### Generate $m$ data points $\{y_i, x_i\}$ satisfying $y_i = x_i^\top \beta$, where $\beta \in \mathbb{R}^n$.

In [1]:
import numpy as np

seed = 431
np.random.seed(seed)


def generate_data(m, n):
    """
    Generate data for a noiseless linear model y = X beta.

    Parameters:
    - m: Number of data points (rows of X).
    - n: Number of features (columns of X and size of beta).

    Returns:
    - X: Random matrix of size m x n.
    - y: Vector of size m obtained using the linear model.
    - beta: Gaussian random vector of size n.
    """

    # Generate random matrix X of size m x n
    X = np.random.rand(m, n)

    # Generate Gaussian random vector beta of size n
    beta = np.random.randn(n)

    # Compute y using the linear model
    y = X @ beta

    return X, y, beta


# Example usage:
# m, n = 100, 5
# X, y, beta = generate_data(m, n)

## Consider the rank deficient case where $m < n$. Show that there are multiple solutions to the linear equation $y = X \beta$.

We generate some data $X, y$ with $m = 50$ and $n = 200$. We can see that $y = X b$ has at least two solutions.

In [2]:
def find_two_solutions_corrected(X, y):
    # Compute the pseudo-inverse of X
    X_pseudo = np.linalg.pinv(X)

    # Compute the minimum norm solution
    beta_min_norm = np.dot(X_pseudo, y)

    # Compute a basis for the null space of X
    _, _, Vt = np.linalg.svd(X)
    null_space_basis = Vt[-(X.shape[1] - np.linalg.matrix_rank(X)) :].T

    # Generate a vector in the null space of X
    arbitrary_coefficients = np.random.rand(null_space_basis.shape[1])
    null_space_vector = np.dot(null_space_basis, arbitrary_coefficients)

    # Compute the arbitrary solution
    beta_arbitrary = beta_min_norm + null_space_vector

    return beta_min_norm, beta_arbitrary


m = 50
n = 200
X, y, beta = generate_data(m, n)
b1, b2 = find_two_solutions_corrected(X, y)

print(
    f"b1 and b2 are solutions to $y = X *beta$ because the residue are {np.linalg.norm(y - X@b1)},{np.linalg.norm(y - X@b2)}"
)

b1 and b2 are solutions to $y = X *beta$ because the residue are 5.520812141926581e-14,6.885540186746371e-14


# Min-norm solution via cvxpy

The mathematical formulation for finding the minimum norm solution $ \mathbf{b} $ subject to the constraint $ y = X \mathbf{b} $ is as follows:


\begin{align*}
\text{minimize} \quad & \lVert \mathbf{b} \rVert_2 \\
\text{subject to} \quad & X \mathbf{b} = y
\end{align*}

Where:
- $ \lVert \mathbf{b} \rVert_2 $ is the Euclidean norm of $ \mathbf{b} $.
- $ X $ is the given matrix.
- $ y $ is the given vector.



Once you run this code in your local environment with CVXPY installed, it will return the minimum norm solution $ \mathbf{b} $ for the equation $ y = X \mathbf{b} $.

In [3]:
import cvxpy as cp


def min_norm_solution_cvxpy(X, y):
    # Define the variable
    b = cp.Variable(X.shape[1])

    # Your codes start here
    problem = cp.Problem(cp.Minimize(cp.norm(b)), [X @ b == y])
    problem.solve()

    return b.value

# Gradient descent starting from zero.

 The gradient descent algorithm updates the parameters iteratively using the gradient of the objective function with respect to the parameters.

For the linear system $ y = X \beta $ and the objective function $J(\beta) = \frac{1}{m} \lVert X \beta - y \rVert_2^2 $, the gradient with respect to $ \beta $ is:
$$
\nabla J(\beta) =\frac{2}{m} X^T (X \beta - y)
$$

Using the gradient descent update rule, we can iteratively update the solution $ \beta $ as:

$$
\beta_{\text{new}} = \beta_{\text{old}} - \alpha \nabla J(\beta)
$$

Where $ \alpha $ is the learning rate. You are going to implement gradient descent, initialized from a zero vector.

  

In [4]:
def gradient_descent(X, y, learning_rate=0.001, num_iterations=10000):
    # Initialize beta as a zero vector
    beta = np.zeros(X.shape[1])
    # Your code starts here, return the final iterate of gradient descnet algorithm, intialize beta as a zero vector

    for i in range(num_iterations):
        grad = 2 / X.shape[0] * X.T @ (X @ beta - y)
        beta = beta - learning_rate * grad

    return beta


def loss_and_gradient(X, y, beta):
    # compute least-squres loss and the norm of gradient
    # Your code starts here
    # compute the loss function $f(\beta) = \frac{1}{2m} \|y - X \beta\|_2^2$. Note that you need to normalize by $m$.
    # compute the gradient of the loss function $\nabla f(\beta) = \frac{1}{m} X^T (X \beta - y)$
    X_beta = X @ beta

    loss = np.linalg.norm(X_beta - y) ** 2 / X.shape[0]
    grad_norm = 2 / X.shape[0] * np.linalg.norm(X.T @ (X_beta - y))

    return loss, grad_norm

Now test thse two functions. We want to show that gradient descent finds the min-norm solution.

In [5]:
m = 50
n = 200
X, y, beta = generate_data(m, n)

beta_min_norm = min_norm_solution_cvxpy(X, y)
beta_gd = gradient_descent(X, y, learning_rate=0.002, num_iterations=50000)
error = np.linalg.norm(beta_gd - beta_min_norm)
if error < 1e-5:
    print(f"the differene between the two solutions are bounded by {error}")
    print(
        "gradient descent with a very small learning rate coincides with the min-norm solution"
    )
    print(
        f"the loss and gradient of loss of GD solution is {loss_and_gradient(X, y, beta_gd)}"
    )
else:
    print(f"the differene between the two solutions are bounded by {error}")
    print("gradient descent does not coincide with min-norm solution")
    print(
        f"the loss and gradient of loss of GD solution is {loss_and_gradient(X, y, beta_gd)}"
    )

the differene between the two solutions are bounded by 8.46207335792476e-09
gradient descent with a very small learning rate coincides with the min-norm solution
the loss and gradient of loss of GD solution is (6.2284596554881515e-18, 1.4775673773399028e-09)


In [8]:
m = 50
n = 200
X, y, beta = generate_data(m, n)


beta_min_norm = min_norm_solution_cvxpy(X, y)
beta_gd2 = gradient_descent(X, y, learning_rate=0.01, num_iterations=50000)
error = np.linalg.norm(beta_gd2 - beta_min_norm)

if error < 1e-5:
    print(f"the differene between the two solutions are bounded by {error}")
    print(
        "gradient descent with a very small learning rate coincides with the min-norm solution"
    )
    print(
        f"the loss and gradient of loss of GD solution is {loss_and_gradient(X, y, beta_gd2)}"
    )
else:
    print(f"the differene between the two solutions are bounded by {error}")
    print("gradient descent does not coincide with min-norm solution")
    print(
        f"the loss and gradient of loss of GD solution is {loss_and_gradient(X, y, beta_gd2)}"
    )

the differene between the two solutions are bounded by 2.36483610505301e-10
gradient descent with a very small learning rate coincides with the min-norm solution
the loss and gradient of loss of GD solution is (3.490222630513036e-28, 1.4475067301903562e-14)
